# Final Project - Group 13
## Project Title: How Covid-19 Policy Influences Violent Crimes 
### People: Zhiwhen Zhu, Hieu Han Nguyen, Gracie Siu 
### Dataset:
- https://www.census.gov/geographies/mapping-files/time-series/geo/carto-boundary-file.html
- https://statepolicies.com/data/library/
- https://cde.ucr.cjis.gov/LATEST/webapp/#/pages/explorer/crime/crime-trend  
- https://apps.bea.gov/regional/histdata/releases/1222gdpstate/index.cfm

### Part 1: Data Crawling

We would like to obtain a dataset of number of crimes committed in a state grouped by a certain type of relationship between the victim and the offender (e.g. spouses, strangers). This is the code we used to query the website's API, downloading our state-level data, and grouping it into a useable .csv. This code took 8 hours to run, so we don't recommend re-running it and did not group this with the "additional analyses" notebook (we want to run all the codes in both notebooks).

In [1]:
import pandas as pd
import requests
key = 'lNBuRRtyECpPMoJRCgfAOcDNXqaDWEvTdbNobxSQ'

In [2]:
states= ("AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DE", "FL", "GA",
              "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD",
              "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ",
              "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI",
              "SC","SD","TN","TX","UT","VT","VA","WA","WV","WI","WY")

In [3]:
ori_data = []
for s in states:
    url = f'https://api.usa.gov/crime/fbi/cde/agency/byStateAbbr/{s}?API_KEY={key}'
    data = requests.get(url)
    ori_data.append(pd.DataFrame(data.json()))

agency = pd.concat(ori_data,axis=0).reset_index(drop=True)

In [4]:
agency

,ori,agency_name,agency_id,state_name,state_abbr,division_name,region_name,region_desc,county_name,agency_type_name,nibrs,nibrs_start_date,latitude,longitude
0,AL0010000,Jefferson County Sheriff's Office,49,Alabama,AL,East South Central,South,Region III,JEFFERSON,County,True,2021-01-01T00:00:00.000Z,33.553444,-86.896536
1,AL0010100,Bessemer Police Department,50,Alabama,AL,East South Central,South,Region III,JEFFERSON,City,True,2006-01-01T00:00:00.000Z,33.396152,-86.95345
2,AL0010200,Birmingham Police Department,51,Alabama,AL,East South Central,South,Region III,"JEFFERSON, SHELBY",City,True,2021-02-01T00:00:00.000Z,33.553444,-86.896536
3,AL0010300,Mountain Brook Police Department,52,Alabama,AL,East South Central,South,Region III,JEFFERSON,City,True,2021-01-01T00:00:00.000Z,33.50124,-86.75607
4,AL0010400,Fairfield Police Department,53,Alabama,AL,East South Central,South,Region III,JEFFERSON,City,False,None,33.553444,-86.896536
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18876,WY0230000,Weston County Sheriff's Office,21933,Wyoming,WY,Mountain,West,Region IV,WESTON,County,True,2021-01-01T00:00:00.000Z,43.846213,-104.57002
18877,WY0230100,Newcastle Police Department,21934,Wyoming,WY,Mountain,West,Region IV,WESTON,City,True,2020-12-01T00:00:00.000Z,43.85474,-104.20272
18878,WY0230200,Upton Police Department,21935,Wyoming,WY,Mountain,West,Region IV,WESTON,City,True,2022-01-01T00:00:00.000Z,43.846213,-104.57002
18879,WYDI05000,Wind River Agency,21936,Wyoming,WY,Mountain,West,Region IV,NOT SPECIFIED,Tribal,True,2021-01-01T00:00:00.000Z,43.055303,-108.605531


In [14]:
relation_data = []
for i, o in enumerate(agency['ori']):
    print(f'Getting agency {i+1} out of {agency.shape[0]}')
    url = f'https://api.usa.gov/crime/fbi/cde/nibrs/agency/{o}/all/victim/relationship?from=2018&to=2022&API_KEY={key}'
    try:
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()['data']  
            for item in data:
                item['ori'] = o  
                expanded_data = {**item, **{'ori': o, 'data_year': item['data_year']}}
                relation_data.append(expanded_data)
        else:
            print(f'Failed to query {o}, Status code: {response.status_code}')
    except Exception as e:
        print(f'Failed to query {o}: {e}')
relation_df = pd.DataFrame(relation_data)
relation_df.reset_index(drop=True, inplace=True)
relation_df['ori_first_two'] = relation_df['ori'].str[:2]
grouped_sum = relation_df.groupby(['ori_first_two','data_year'])[[
    'Acquaintance', 'Babysittee', 'Boyfriend/Girlfriend',
    'Child of Boyfriend/Girlfriend', 'Child', 'Employee', 'Employer',
    'Friend', 'Grandchild', 'Grandparent', 'Homosexual Relationship',
    'In-Law', 'Neighbor', 'Other Family Member', 'Otherwise Known',
    'Parent', 'Relationship Unknown', 'Sibling', 'Stepchild', 'Spouse',
    'Stepparent', 'Stepsibling', 'Stranger', 'Offender', 'Ex Spouse',
    'Common Law Spouse'
]].sum()

Getting agency 1 out of 18881
Getting agency 2 out of 18881
Getting agency 3 out of 18881
Getting agency 4 out of 18881
Getting agency 5 out of 18881
Getting agency 6 out of 18881
Getting agency 7 out of 18881
Getting agency 8 out of 18881
Getting agency 9 out of 18881
Getting agency 10 out of 18881
Getting agency 11 out of 18881
Getting agency 12 out of 18881
Getting agency 13 out of 18881
Getting agency 14 out of 18881
Getting agency 15 out of 18881
Getting agency 16 out of 18881
Getting agency 17 out of 18881
Getting agency 18 out of 18881
Getting agency 19 out of 18881
Getting agency 20 out of 18881
Getting agency 21 out of 18881
Getting agency 22 out of 18881
Getting agency 23 out of 18881
Getting agency 24 out of 18881
Getting agency 25 out of 18881
Getting agency 26 out of 18881
Getting agency 27 out of 18881
Getting agency 28 out of 18881
Getting agency 29 out of 18881
Getting agency 30 out of 18881
Getting agency 31 out of 18881
Getting agency 32 out of 18881
Getting agency 33

Getting agency 261 out of 18881
Getting agency 262 out of 18881
Getting agency 263 out of 18881
Getting agency 264 out of 18881
Getting agency 265 out of 18881
Getting agency 266 out of 18881
Getting agency 267 out of 18881
Getting agency 268 out of 18881
Getting agency 269 out of 18881
Getting agency 270 out of 18881
Getting agency 271 out of 18881
Getting agency 272 out of 18881
Getting agency 273 out of 18881
Getting agency 274 out of 18881
Getting agency 275 out of 18881
Getting agency 276 out of 18881
Getting agency 277 out of 18881
Getting agency 278 out of 18881
Getting agency 279 out of 18881
Getting agency 280 out of 18881
Getting agency 281 out of 18881
Getting agency 282 out of 18881
Getting agency 283 out of 18881
Getting agency 284 out of 18881
Getting agency 285 out of 18881
Getting agency 286 out of 18881
Getting agency 287 out of 18881
Getting agency 288 out of 18881
Getting agency 289 out of 18881
Getting agency 290 out of 18881
Getting agency 291 out of 18881
Getting 

Getting agency 518 out of 18881
Getting agency 519 out of 18881
Getting agency 520 out of 18881
Getting agency 521 out of 18881
Getting agency 522 out of 18881
Getting agency 523 out of 18881
Getting agency 524 out of 18881
Getting agency 525 out of 18881
Getting agency 526 out of 18881
Getting agency 527 out of 18881
Getting agency 528 out of 18881
Getting agency 529 out of 18881
Getting agency 530 out of 18881
Getting agency 531 out of 18881
Getting agency 532 out of 18881
Getting agency 533 out of 18881
Getting agency 534 out of 18881
Getting agency 535 out of 18881
Getting agency 536 out of 18881
Getting agency 537 out of 18881
Getting agency 538 out of 18881
Getting agency 539 out of 18881
Getting agency 540 out of 18881
Getting agency 541 out of 18881
Getting agency 542 out of 18881
Getting agency 543 out of 18881
Getting agency 544 out of 18881
Getting agency 545 out of 18881
Getting agency 546 out of 18881
Getting agency 547 out of 18881
Getting agency 548 out of 18881
Getting 

Getting agency 775 out of 18881
Getting agency 776 out of 18881
Getting agency 777 out of 18881
Getting agency 778 out of 18881
Getting agency 779 out of 18881
Getting agency 780 out of 18881
Getting agency 781 out of 18881
Getting agency 782 out of 18881
Getting agency 783 out of 18881
Getting agency 784 out of 18881
Getting agency 785 out of 18881
Getting agency 786 out of 18881
Getting agency 787 out of 18881
Getting agency 788 out of 18881
Getting agency 789 out of 18881
Getting agency 790 out of 18881
Getting agency 791 out of 18881
Getting agency 792 out of 18881
Getting agency 793 out of 18881
Getting agency 794 out of 18881
Getting agency 795 out of 18881
Getting agency 796 out of 18881
Getting agency 797 out of 18881
Getting agency 798 out of 18881
Getting agency 799 out of 18881
Getting agency 800 out of 18881
Getting agency 801 out of 18881
Getting agency 802 out of 18881
Getting agency 803 out of 18881
Getting agency 804 out of 18881
Getting agency 805 out of 18881
Getting 

Getting agency 1031 out of 18881
Getting agency 1032 out of 18881
Getting agency 1033 out of 18881
Getting agency 1034 out of 18881
Getting agency 1035 out of 18881
Getting agency 1036 out of 18881
Getting agency 1037 out of 18881
Getting agency 1038 out of 18881
Getting agency 1039 out of 18881
Getting agency 1040 out of 18881
Getting agency 1041 out of 18881
Getting agency 1042 out of 18881
Getting agency 1043 out of 18881
Getting agency 1044 out of 18881
Getting agency 1045 out of 18881
Getting agency 1046 out of 18881
Getting agency 1047 out of 18881
Getting agency 1048 out of 18881
Getting agency 1049 out of 18881
Getting agency 1050 out of 18881
Getting agency 1051 out of 18881
Getting agency 1052 out of 18881
Getting agency 1053 out of 18881
Getting agency 1054 out of 18881
Getting agency 1055 out of 18881
Getting agency 1056 out of 18881
Getting agency 1057 out of 18881
Getting agency 1058 out of 18881
Getting agency 1059 out of 18881
Getting agency 1060 out of 18881
Getting ag

...

Getting agency 17708 out of 18881
Getting agency 17709 out of 18881
Getting agency 17710 out of 18881
Getting agency 17711 out of 18881
Getting agency 17712 out of 18881
Getting agency 17713 out of 18881
Getting agency 17714 out of 18881
Getting agency 17715 out of 18881
Getting agency 17716 out of 18881
Getting agency 17717 out of 18881
Getting agency 17718 out of 18881
Getting agency 17719 out of 18881
Getting agency 17720 out of 18881
Getting agency 17721 out of 18881
Getting agency 17722 out of 18881
Getting agency 17723 out of 18881
Getting agency 17724 out of 18881
Getting agency 17725 out of 18881
Getting agency 17726 out of 18881
Getting agency 17727 out of 18881
Getting agency 17728 out of 18881
Getting agency 17729 out of 18881
Getting agency 17730 out of 18881
Getting agency 17731 out of 18881
Getting agency 17732 out of 18881
Getting agency 17733 out of 18881
Getting agency 17734 out of 18881
Getting agency 17735 out of 18881
Getting agency 17736 out of 18881
Getting agency

Getting agency 17949 out of 18881
Getting agency 17950 out of 18881
Getting agency 17951 out of 18881
Getting agency 17952 out of 18881
Getting agency 17953 out of 18881
Getting agency 17954 out of 18881
Getting agency 17955 out of 18881
Getting agency 17956 out of 18881
Getting agency 17957 out of 18881
Getting agency 17958 out of 18881
Getting agency 17959 out of 18881
Getting agency 17960 out of 18881
Getting agency 17961 out of 18881
Getting agency 17962 out of 18881
Getting agency 17963 out of 18881
Getting agency 17964 out of 18881
Getting agency 17965 out of 18881
Getting agency 17966 out of 18881
Getting agency 17967 out of 18881
Getting agency 17968 out of 18881
Getting agency 17969 out of 18881
Getting agency 17970 out of 18881
Getting agency 17971 out of 18881
Getting agency 17972 out of 18881
Getting agency 17973 out of 18881
Getting agency 17974 out of 18881
Getting agency 17975 out of 18881
Getting agency 17976 out of 18881
Getting agency 17977 out of 18881
Getting agency

Getting agency 18190 out of 18881
Getting agency 18191 out of 18881
Getting agency 18192 out of 18881
Getting agency 18193 out of 18881
Getting agency 18194 out of 18881
Getting agency 18195 out of 18881
Getting agency 18196 out of 18881
Getting agency 18197 out of 18881
Getting agency 18198 out of 18881
Getting agency 18199 out of 18881
Getting agency 18200 out of 18881
Getting agency 18201 out of 18881
Getting agency 18202 out of 18881
Getting agency 18203 out of 18881
Getting agency 18204 out of 18881
Getting agency 18205 out of 18881
Getting agency 18206 out of 18881
Getting agency 18207 out of 18881
Getting agency 18208 out of 18881
Getting agency 18209 out of 18881
Getting agency 18210 out of 18881
Getting agency 18211 out of 18881
Getting agency 18212 out of 18881
Getting agency 18213 out of 18881
Getting agency 18214 out of 18881
Getting agency 18215 out of 18881
Getting agency 18216 out of 18881
Getting agency 18217 out of 18881
Getting agency 18218 out of 18881
Getting agency

Getting agency 18431 out of 18881
Getting agency 18432 out of 18881
Getting agency 18433 out of 18881
Getting agency 18434 out of 18881
Getting agency 18435 out of 18881
Getting agency 18436 out of 18881
Getting agency 18437 out of 18881
Getting agency 18438 out of 18881
Getting agency 18439 out of 18881
Getting agency 18440 out of 18881
Getting agency 18441 out of 18881
Getting agency 18442 out of 18881
Getting agency 18443 out of 18881
Getting agency 18444 out of 18881
Getting agency 18445 out of 18881
Getting agency 18446 out of 18881
Getting agency 18447 out of 18881
Getting agency 18448 out of 18881
Getting agency 18449 out of 18881
Getting agency 18450 out of 18881
Getting agency 18451 out of 18881
Getting agency 18452 out of 18881
Getting agency 18453 out of 18881
Getting agency 18454 out of 18881
Getting agency 18455 out of 18881
Getting agency 18456 out of 18881
Getting agency 18457 out of 18881
Getting agency 18458 out of 18881
Getting agency 18459 out of 18881
Getting agency

Getting agency 18672 out of 18881
Getting agency 18673 out of 18881
Getting agency 18674 out of 18881
Getting agency 18675 out of 18881
Getting agency 18676 out of 18881
Getting agency 18677 out of 18881
Getting agency 18678 out of 18881
Getting agency 18679 out of 18881
Getting agency 18680 out of 18881
Getting agency 18681 out of 18881
Getting agency 18682 out of 18881
Getting agency 18683 out of 18881
Getting agency 18684 out of 18881
Getting agency 18685 out of 18881
Getting agency 18686 out of 18881
Getting agency 18687 out of 18881
Getting agency 18688 out of 18881
Getting agency 18689 out of 18881
Getting agency 18690 out of 18881
Getting agency 18691 out of 18881
Getting agency 18692 out of 18881
Getting agency 18693 out of 18881
Getting agency 18694 out of 18881
Getting agency 18695 out of 18881
Getting agency 18696 out of 18881
Getting agency 18697 out of 18881
Getting agency 18698 out of 18881
Getting agency 18699 out of 18881
Getting agency 18700 out of 18881
Getting agency

In [15]:
grouped_sum

Acquaintance  Babysittee  Boyfriend/Girlfriend  \
ori_first_two data_year                                                   
AK            2018                  0           0                     0   
              2019                  0           0                     0   
              2020                  8           0                     4   
              2021               1540           9                  1907   
              2022               1083           4                  1614   
...                               ...         ...                   ...   
WY            2018                  1           0                     0   
              2019                124           3                   130   
              2020                363           6                   249   
              2021               1794          18                   971   
              2022               1855          14                   955   

                         Child of Boyfriend/Girlfriend  Child  Employee  \
ori_first_two data_year                                                   
AK            2018                                   0      0         0   
              2019                                   0      1         0   
              2020                                   2      0         0   
              2021                                  53    360        11   
              2022                                  43    330        18   
...                                                ...    ...       ...   
WY            2018                                   0      0         0   
              2019                                   7     19         2   
              2020                                   8     48         8   
              2021                                  28    348        55   
              2022                                  34    351        28   

                         Employer  Friend  Grandchild  Grandparent  ...  \
ori_first_two data_year                                             ...   
AK            2018              0       0           0            0  ...   
              2019              0       0           0            0  ...   
              2020              0       1           0            0  ...   
              2021             95     364          22           73  ...   
              2022             27     342          34           68  ...   
...                           ...     ...         ...          ...  ...   
WY            2018              0       0           0            0  ...   
              2019              6      27           0            6  ...   
              2020              6      76           5            6  ...   
              2021             41     334          31           50  ...   
              2022             32     327          33           61  ...   

                         Relationship Unknown  Sibling  Stepchild  Spouse  \
ori_first_two data_year                                                     
AK            2018                          0        0          1       0   
              2019                          2        0          0       0   
              2020                          7        1          0       1   
              2021                        865      520         45     657   
              2022                        798      521         48     618   
...                                       ...      ...        ...     ...   
WY            2018                          0        0          0       0   
              2019                         29       24          2      38   
              2020                         86       42         11     159   
              2021                        479      224         58     479   
              2022                        354      220         57     507   

                         Stepparent  Stepsibling  Stranger  Offender  \
ori_first_two data_year

In [19]:
grouped_sum.to_csv('relationship_2018-2022.csv', index=True)

## References
- https://cde.ucr.cjis.gov/LATEST/webapp/#/pages/explorer/crime/crime-trend
- https://api.data.gov/
- https://www.justice.gov/developer
- https://cde.ucr.cjis.gov/LATEST/webapp/#/pages/docApi
- https://andrewpwheeler.com/2023/07/29/downloading-police-employment-trends-from-the-fbi-data-explorer/